In [1]:
import pandas as pd
import numpy as np
from analysis.utils import process_categorical

In [2]:
df = pd.read_csv('../data/df_clean.csv', index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17796 entries, 0 to 18357
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17796 non-null  object 
 1   League                    17796 non-null  object 
 2   NationalLeagueLevel       17796 non-null  object 
 3   Club                      17796 non-null  object 
 4   No                        17511 non-null  float64
 5   Name                      17796 non-null  object 
 6   Value                     17796 non-null  float64
 7   HighestValue              17796 non-null  float64
 8   Age                       17794 non-null  float64
 9   Height                    16717 non-null  float64
 10  Nationality               17796 non-null  object 
 11  Foot                      16124 non-null  object 
 12  Position                  17796 non-null  object 
 13  Consultancy               14524 non-null  object 
 14  Suppli

In [4]:
df = df.loc[df['Position'] == 'Torwart']

In [5]:
features = ['LeagueCountry', 'NationalLeagueLevel', 'InternationalTeam', 'League', 'Club', 'Age', 'ClubSince', 'ActiveInternational', 'FormerInternational', 'InternationalGames', 'Trending', 'Games', 'PointsPerGame', 'Cards', 'Minutes', 'TwCleanSheets', 'TwGoalsConceded', 'TwPenaltySavesQuote', 'Value']

df = df[features]

In [6]:
numeric_columns = df.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    print(f"Column: {col}, NaN values: {numeric_columns[col].isna().sum()}")

Column: Age, NaN values: 0
Column: ClubSince, NaN values: 10
Column: ActiveInternational, NaN values: 0
Column: FormerInternational, NaN values: 0
Column: InternationalGames, NaN values: 863
Column: Trending, NaN values: 0
Column: Games, NaN values: 568
Column: PointsPerGame, NaN values: 995
Column: Cards, NaN values: 568
Column: Minutes, NaN values: 996
Column: TwCleanSheets, NaN values: 568
Column: TwGoalsConceded, NaN values: 568
Column: TwPenaltySavesQuote, NaN values: 287
Column: Value, NaN values: 0


In [7]:
df = df.dropna(subset=['Age', 'ClubSince', 'Games', 'PointsPerGame', 'Cards', 'Minutes', 'TwCleanSheets', 'TwGoalsConceded', 'TwPenaltySavesQuote'])

In [8]:
df = df.fillna({'InternationalGames': 0})

In [9]:
numeric_columns = df.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    print(f"Column: {col}, NaN values: {numeric_columns[col].isna().sum()}")

Column: Age, NaN values: 0
Column: ClubSince, NaN values: 0
Column: ActiveInternational, NaN values: 0
Column: FormerInternational, NaN values: 0
Column: InternationalGames, NaN values: 0
Column: Trending, NaN values: 0
Column: Games, NaN values: 0
Column: PointsPerGame, NaN values: 0
Column: Cards, NaN values: 0
Column: Minutes, NaN values: 0
Column: TwCleanSheets, NaN values: 0
Column: TwGoalsConceded, NaN values: 0
Column: TwPenaltySavesQuote, NaN values: 0
Column: Value, NaN values: 0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031 entries, 0 to 18333
Columns: 124 entries, Age to NationalLeagueLevel_other
dtypes: float64(11), int64(3), uint8(110)
memory usage: 231.6 KB
200000.0


In [11]:
df = process_categorical(df, 'InternationalTeam', 5)
df = process_categorical(df, 'League')
df = process_categorical(df, 'Club')
df = process_categorical(df, 'LeagueCountry', 5)
df = process_categorical(df, 'NationalLeagueLevel', 5)

In [12]:
df.to_csv('../data/df_extensive_model_tw.csv')